In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew, LLM
import os

# Create Groq-backed CrewAI LLM
llm = LLM(
    model="groq/llama-3.1-8b-instant",
    temperature=0.7,
    api_key=os.environ["GROQ_API_KEY"]
)

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [4]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool],
    llm=llm
)

In [5]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [6]:
# The Process class helps to delegate the workflow to the Agents (kind of like a Manager at work)

from crewai import Crew, Process

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=llm,
    process=Process.hierarchical,
    verbose=True
)

In [7]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7701384c-8ae3-4e69-83cc-9fb3adb41f68                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling     │
│  and machine learning to identify trends and predict market movements.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Thought: Action: Read website content                                                                          │
│                                                                                                                 │
│  Using Tool: Read website content                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "website_url": "https://finance.yahoo.com/quote/AAPL/history?p=AAPL"                                         │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The following text is scraped website content:                                                                 │
│  Apple Inc. (AAPL) Stock Historical Prices & Data - Yahoo Finance                                               │
│  Oops, something went wrong Skip to navigation Skip to main content Skip to right column News Today's news US   │
│  Politics 2025 Election World Weather Climate change Health Wellness Mental health Sexual health Dermatology    │
│  Oral health Hair loss Foot health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements  │
│  Fitness Equipment Exercise Women's health Sleep Healthy aging Hearing Mobility Science Originals The 360       │
│  Newsletters Games Life Health Wellness Nutrition Fitness Healthy aging Mental health Sleep Your body           │
│  Dermatology Children's health Foot health Hair loss Hearing Oral health Sexual health Women's health           │
│  Conditions Cardiovascular health Digestive health Endocrine system Parenting Family health So mini ways Style  │
│  and beauty It Figures Unapologetically Horoscopes Shopping Style Accessories Clothing Luggage Shoes Beauty     │
│  Hair Makeup Skincare Sunscreen Health Dental Fitness Hair loss Hearing aids Mental health Mobility Nutrition   │
│  Personal care Sleep Women's health Home and garden Bedding Cleaning Gardening Kitchen Outdoor Pets Tech        │
│  Accessories Audio Auto Computers Phones Smart home TVs Gift ideas Best Advent calendars 2025 Best Christmas    │
│  gifts Best White Elephant gifts Best Secret Santa gifts Best gifts for men Best gift cards Best gifts for mom  │
│  Best gifts for teens Stores Amazon Best Buy Home Depot Macy's Nordstrom Target Walmart Wayfair Shopping        │
│  Guides Best non-toxic cutting boards Best cordless stick vacuums Best water bottles Best vacuums Best air      │
│  fryers Best luggage Best jeans Best bras Best nontoxic nail polish Best under eye cream Best skin tightening   │
│  creams Best coffee makers Deals Black Friday 2025 Black Friday deals Amazon Black Friday Target Black Friday   │
│  Walmart Black Friday Black Friday Tech deals Best Black Friday deals Food Travel Autos EV & Future Tech        │
│  Classic & Collector Deals & Buying Guides Ownership Safety & Recalls Po...                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: a86d20fb-9b3c-413b-9e89-a4d26c5d94b0                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for      │
│  model `llama-3.1-8b-instant` in organization `org_01k9y8h1xce208nebm8d63gja1` service tier `on_demand` on      │
│  tokens per minute (TPM): Limit 6000, Requested 11706, please reduce your message size and try again. Need      │
│  more tokens? Upgrade to Dev Tier today at                                                                      │
│  https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Request too large for model `llama-3.1-8b-instant` in organization `org_01k9y8h1xce208nebm8d63gja1` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 11706, please reduce your message size and try again. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x119e5b2d0>: Failed to establish a new connection: [Errno 61] Connection refused'))
HTTPSConnectionPool(host='telemetry.crewai.com', port=4319): Max retries exceeded with url: /v1/traces (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11e6f8090>: Failed to establish a new connection: [Errno 61] Connection refused'))


In [ ]:
# example of designed asynchronous execution with asyncio

# import asyncio

# async def run_pipeline():
#     # 1) run first agent
#     res1 = await agent1.kickoff_async("do step 1")

#     # 2) run second & third concurrently
#     res2, res3 = await asyncio.gather(
#         agent2.kickoff_async({"from_step_1": res1.raw}),
#         agent3.kickoff_async({"from_step_1": res1.raw}),
#     )

#     # 3) run fourth after both are done
#     res4 = await agent4.kickoff_async(
#         {"from_step_2": res2.raw, "from_step_3": res3.raw}
#     )
#     return res4

# final = asyncio.run(run_pipeline())
# print(final.raw)
